In [1]:
import os

import numpy as np
import pandas as pd
import polars as pl

from scipy.optimize import minimize, Bounds


In [2]:
MIN_INVESTMENT = 0
MAX_INVESTMENT = 2

In [3]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import numpy as np
import pandas as pd
import pandas.api.types

MIN_INVESTMENT = 0
MAX_INVESTMENT = 2


class ParticipantVisibleError(Exception):
    pass


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    """
    Calculates a custom evaluation metric (volatility-adjusted Sharpe ratio).

    This metric penalizes strategies that take on significantly more volatility
    than the underlying market.

    Returns:
        float: The calculated adjusted Sharpe ratio.
    """

    if not pandas.api.types.is_numeric_dtype(submission['prediction']):
        raise ParticipantVisibleError('Predictions must be numeric')

    solution = solution
    solution['position'] = submission['prediction']

    if solution['position'].max() > MAX_INVESTMENT:
        raise ParticipantVisibleError(f'Position of {solution["position"].max()} exceeds maximum of {MAX_INVESTMENT}')
    if solution['position'].min() < MIN_INVESTMENT:
        raise ParticipantVisibleError(f'Position of {solution["position"].min()} below minimum of {MIN_INVESTMENT}')

    solution['strategy_returns'] = solution['risk_free_rate'] * (1 - solution['position']) + solution['position'] * solution['forward_returns']

    # Calculate strategy's Sharpe ratio
    strategy_excess_returns = solution['strategy_returns'] - solution['risk_free_rate']
    strategy_excess_cumulative = (1 + strategy_excess_returns).prod()
    strategy_mean_excess_return = (strategy_excess_cumulative) ** (1 / len(solution)) - 1
    strategy_std = solution['strategy_returns'].std()

    trading_days_per_yr = 252
    if strategy_std == 0:
        raise ParticipantVisibleError('Division by zero, strategy std is zero')
    sharpe = strategy_mean_excess_return / strategy_std * np.sqrt(trading_days_per_yr)
    strategy_volatility = float(strategy_std * np.sqrt(trading_days_per_yr) * 100)

    # Calculate market return and volatility
    market_excess_returns = solution['forward_returns'] - solution['risk_free_rate']
    market_excess_cumulative = (1 + market_excess_returns).prod()
    market_mean_excess_return = (market_excess_cumulative) ** (1 / len(solution)) - 1
    market_std = solution['forward_returns'].std()

    market_volatility = float(market_std * np.sqrt(trading_days_per_yr) * 100)

    if market_volatility == 0:
        raise ParticipantVisibleError('Division by zero, market std is zero')

    # Calculate the volatility penalty
    excess_vol = max(0, strategy_volatility / market_volatility - 1.2) if market_volatility > 0 else 0
    vol_penalty = 1 + excess_vol

    # Calculate the return penalty
    return_gap = max(
        0,
        (market_mean_excess_return - strategy_mean_excess_return) * 100 * trading_days_per_yr,
    )
    return_penalty = 1 + (return_gap**2) / 100

    # Adjust the Sharpe ratio by the volatility and return penalty
    adjusted_sharpe = sharpe / (vol_penalty * return_penalty)
    return min(float(adjusted_sharpe), 1_000_000)


In [4]:
train = pd.read_csv("/kaggle/input/hull-tactical-market-prediction/train.csv", index_col="date_id")

In [5]:
solution = train.loc[8810:8990, ["forward_returns", "risk_free_rate"]]

def fun(x):
    submission = pd.DataFrame({"prediction": x}, index=solution.index)
    return - score(solution, submission, None)

In [6]:
res = minimize(
    fun,
    x0=np.full(solution.shape[0], 0.1),
    method="Powell",
    bounds=Bounds(lb=MIN_INVESTMENT, ub=MAX_INVESTMENT),
    tol=1e-9
)

In [7]:
solution["prediction"] = res.x
solution

,forward_returns,risk_free_rate,position,strategy_returns,prediction
date_id,,,,,
8810,0.005988,0.000167,9.837409e-02,0.000740,9.837409e-02
8811,0.011115,0.000167,5.230460e-02,0.000740,5.230460e-02
8812,0.000067,0.000167,4.547755e-10,0.000167,4.547755e-10
8813,-0.010526,0.000167,4.547755e-10,0.000167,4.547755e-10
8814,-0.011412,0.000166,4.547755e-10,0.000166,4.547755e-10
...,...,...,...,...,...
8986,0.002312,0.000156,2.708806e-01,0.000740,2.708806e-01
8987,0.002891,0.000156,2.133648e-01,0.000740,2.133648e-01
8988,0.008310,0.000156,7.161083e-02,0.000740,7.161083e-02


In [8]:
import kaggle_evaluation.default_inference_server

def predict(test: pl.DataFrame) -> pl.DataFrame:
    test_df = test.to_pandas().set_index("date_id")
    test_df = test_df.join(solution)
    test = test.with_columns(
        prediction = test_df["prediction"].fillna(0.0).values
    )
    return test


inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
    inference_server.serve()
else:
    inference_server.run_local_gateway(("/kaggle/input/hull-tactical-market-prediction/",))